## Exponea Data Analytics Challenge 

**Author:** Štefan Konečný

**Email:** konecny.mokum@gmail.com

**Date:** 15 February 2017

### Preview of the Data

Let us have a brief look at the data:

In [152]:
%matplotlib inline

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display

startup_df = pd.read_csv("Startup_Data.csv")
display(startup_df.head())

print "Number of companies "+str(startup_df.index.size)

startup_df[["Company_Name", "Dependent-Company Status"]].groupby( ["Dependent-Company Status"] ).count()

import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
#di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,...,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,Company1,Success,No Info,No Info,-1.0,Video distribution,NaN,operation,KPCB Holdings|Draper Fisher Jurvetson (DFJ)|Kl...,3.0,...,0,0,0,0,0,0,0,0,0,0
1,Company2,Success,2011,3,125.0,NaN,Market Research|Marketing|Crowdfunding,"Marketing, sales",NaN,NaN,...,8.823529412,21.76470588,10.88235294,2.941176471,0,0,0,0,0,8
2,Company3,Success,2011,3,455.0,Event Data Analytics API,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,...,3.846153846,17.09401709,9.401709402,0,2.777777778,0,0,0,0,9
3,Company4,Success,2009,5,-99.0,The most advanced analytics for mobile,Mobile|Analytics,Marketing & Sales,Michael Birch|Max Levchin|Sequoia Capital|Keit...,45.0,...,0,0,0,0,0,0,0,0,0,5
4,Company5,Success,2010,4,496.0,The Location-Based Marketing Platform,Analytics|Marketing|Enterprise Software,Marketing & Sales,DFJ Frontier|Draper Nexus Ventures|Gil Elbaz|A...,39.0,...,0,0,0,0,0,0,0,0,0,6


Number of companies 472


We have 472 different companies and 116 varaibles describing each.

Suprisingly there are almost twice as many (305) sucessful companies as failed ones (167). This is of course **VERY SUSPICIOUS** and likely an indication of a synthetic data set. In real life most companies fail.

More importantly, I have to be very careful to **AVOID OVERFITTING** since I have much more data on successful companies the on failed ones.

### My approach

1. Divide companies in sucessfull(S) and failed(F) ones
1. Create normalized histograms for each variable
1. Detect signficicant differences between histograms for S and F companies 

Because the there are much more S companies then F companies I will use normalized histograms for comparing variables. I make sure that:

1. Bin boundaries are the same for S and F
1. Each bins contains the percentage and not a count

In [2]:
success_df = startup_df[startup_df["Dependent-Company Status"]=="Success"].reset_index()
failed_df = startup_df[startup_df["Dependent-Company Status"]=="Failed"].reset_index()

df_sf = (success_df,failed_df)

def normalize_double_hist(data, bins = 20, h_range = None):
    if (h_range == None):
        min_val = min(data[0].min(),data[1].min())
        max_val = max(data[0].max(),data[1].max())
    else:
        min_val = h_range[0]
        max_val = h_range[1]
    
    # if there are both + and - values center on 0, 0 has often a special meaning 
    # so it sholdn't be in a centre of a bin    
    if (min_val * max_val) < 0:
        max_val = max(abs(min_val),max_val)
        min_val = -max_val
        
    #get histograms
    hist0, bin_edges = np.histogram(data[0], bins, range =  (min_val,max_val))
    hist1, _ = np.histogram(data[1], bins, range =  (min_val,max_val))
    
    #normalize histograms
    n_hist0 = hist0.astype(np.float32) / hist0.sum()
    n_hist1 = hist1.astype(np.float32) / hist1.sum()
    
    return (bin_edges,n_hist0,n_hist1) 

### Preprocessing variables

The function above works on numerical data only. Many variables need some pre processing.

1. **Numerical variables**
   * Can ocassionally contain a string (e.g. 'No value')
   * If there are both negative and positive value, 0 should be a border of histogram bins

2. **Categorical variables**
   * Fall into a small number of categories (e.g. 'yes'/'no')
   * Each histogram bin corresponds to one histogram bin   
   * The values can be case sensitive (e.g. 'yes'/'Yes'/'YES')

3. **Other variables**
   * Unique strings ('Company_Name')
   * Set of values ('Industry of company')

We can automatically divide variables into those categories.

In [3]:
cat_var = []
lower_var = []
num_var = []
other_var = []

def split_variables(index, data = df_sf, cv = cat_var, lv = lower_var, nv = num_var, ov =other_var):
    s_col = data[0].iloc[:,index].dropna()
    f_col = data[1].iloc[:,index].dropna()
    
    t_size = float(s_col.size+f_col.size) 
    
    #unique vals
    unique_vals = np.append(s_col.unique(),f_col.unique())
    unique_vals = np.unique(unique_vals)
    
    u_size = float(unique_vals.size)
    
    #probably categorical
    if ((u_size / t_size) < 0.05):
        cv += [index]
        
        #try cat to lower
        l_vals = pd.Series(unique_vals).apply(lambda x: str(x).lower())
        l_vals = np.unique(l_vals)
        l_size = float(l_vals.size)
        
        #if cast made a difference remember this
        if (u_size > l_size):
            lv += [index]
    else:
    #try numbers
        num_vals = pd.Series(unique_vals).apply(lambda x: pd.to_numeric(x, errors='coerce'))
        num_vals = num_vals.dropna()
        
        n_size = float(num_vals.size)
        
        #if the majority is numbers
        if ((n_size / u_size) > 0.9):
            nv += [index]
        #well something else
        else:
            ov += [index]

for i in range(1,117):
    split_variables(i)            
            
print cat_var 
print lower_var
print num_var
print other_var

[2, 3, 4, 12, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 70, 71, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 97, 100, 101, 112, 116]
[12, 26, 39, 40]
[5, 10, 11, 15, 23, 66, 72, 74, 88, 94, 95, 96, 98, 99, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115]
[1, 6, 7, 8, 9, 13, 14, 16, 62]


In [124]:
def survey_column(index, data = df_sf, first_n = 20, bins = 50, force_num = False,\
                  cat = False, ratio = 2, min_p = 0.05, drop_p = 0.1,\
                  print_out = False, draw_hist = False, print_uvals = False):
    
    name = data[0].columns[index]
    
    if print_out:
        print str(index)+" "+ name
    
    s_col = data[0].iloc[:,index]
    f_col = data[1].iloc[:,index]
    
    s_size = s_col.size
    f_size = f_col.size
    
    #show first size rows
    ccol_df = pd.concat([s_col.iloc[0:first_n,],f_col.iloc[0:first_n,]], axis =1)
    #rename columns
    ccol_df.columns =["S:"+name,"F:"+name]
    
    if print_out:
        display(ccol_df)

    s_col = s_col.dropna()
    f_col = f_col.dropna()
    
    if (cat):
        unique_vals = np.append(s_col.unique(),f_col.unique())
        unique_vals = np.unique(unique_vals)
        unique_vals = np.sort(unique_vals)
        
        # print out first nr_vals
        if print_out:
            if unique_vals.size > first_n:
                print unique_vals[0:first_n]
        elif print_uvals:
            print str(index)+" "+ name
            print unique_vals
                
        #cast categories into indexes
        s_col = s_col.apply(lambda x: np.searchsorted(unique_vals,x))
        f_col = f_col.apply(lambda x: np.searchsorted(unique_vals,x))


    #force casting into numbers
    elif force_num:
        s_col = s_col.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        f_col = f_col.apply(lambda x: pd.to_numeric(x, errors='coerce'))
  
        s_col = s_col.dropna()
        f_col = f_col.dropna()
        
    
    s_dropped =(s_size-s_col.size)/float(s_size)
    f_dropped =(f_size-f_col.size)/float(f_size)
    
    if print_out:
        print "Dropped values S/F: {0:2.2f}% / {1:2.2f}%".format(s_dropped*100, f_dropped*100)
    
    #do df summaries
    
    cdes_df = pd.concat([s_col.describe(),f_col.describe()], axis =1)
    #rename columns
    cdes_df.columns =["S:"+name,"F:"+name]
    
    #count how many values there were before preprocessing (before removing weird valeus)
    f_line = pd.DataFrame({"S:"+name: [len(success_df.index)],"F:"+name:[len(failed_df.index)]}, index = ["t_count"])
    
    #prepend fline
    cdes_df = pd.concat([f_line,cdes_df])
    
    if print_out:
        display(cdes_df)

    #get histograms with categorical variables # bins = # categories
    if (cat):
        b, nh_s, nh_f =normalize_double_hist((s_col,f_col),\
                                       bins = unique_vals.size)
    else:
        b, nh_s, nh_f =normalize_double_hist((s_col,f_col),\
                                       bins = bins)
    
    if draw_hist:
        fig, ax = plt.subplots(2,2, figsize=(15,6))

        max_hist = max (nh_s.max(), nh_f.max())

        step = b[1] - b[0]

        scale = 1.1

        ax[0,0].bar(b[:-1], nh_s, width= step, color='green')    
        ax[0,0].set_title("S:"+name)
        ax[0,0].set_xlim([b[0]*scale,b[-1]*scale])
        ax[0,0].set_ylim([0,max_hist])

        ax[0,1].bar(b[:-1], nh_f, width= step, color='red')
        ax[0,1].set_title("F:"+name)
        ax[0,1].set_xlim([b[0]*scale,b[-1]*scale])
        ax[0,1].set_ylim([0,max_hist*scale])

        s_less = nh_s *(nh_s <= nh_f)
        ax[1,0].bar(b[:-1], nh_s, width= step, color='green')
        ax[1,0].bar(b[:-1], nh_f, width= step, color='red')
        ax[1,0].bar(b[:-1], s_less, width= step, color='green')
        ax[1,0].set_title("Both")
        ax[1,0].set_xlim([b[0]*scale,b[-1]*scale])
        ax[1,0].set_ylim([0,max_hist*scale])

#     alternative plotting
#     s_diff = nh_s - nh_f
#     ax[1,1].bar(b[:-1], s_diff, width= step, color='orange')
#     ax[1,1].set_title("Diff")
#     ax[1,1].set_xlim([b[0]*scale,b[-1]*scale])
#     ax[1,1].set_ylim([s_diff.min()*scale,s_diff.max()*scale])
    
#     s_more = (nh_s > nh_f)    
#     f_less_val = s_more * nh_f;
#     f_less_val_neg = f_less_val * -1;
    
    
#     ax[1,1].bar(b[:-1], f_less_val_neg, width= step, color='blue')
#     ax[1,1].set_ylim([min(s_diff.min(),f_less_val_neg.min())*scale,s_diff.max()*scale])
    
    
    #calculate ratio between success and fialure
    sf_ratio = nh_s/nh_f
    # remove inf, inf nan
    sf_ratio[np.logical_not(np.isfinite(sf_ratio))] = 0
    
    
    #ignore entries below ratio
    sf_ratio[(sf_ratio<ratio)] =  0
    #ignore entries not frequent enough
    sf_ratio[(nh_s <= min_p)] =  0
    
    #only sucessess, no fialures in the bin
    s_pos = np.copy(nh_s)
    
    s_pos[nh_f !=0] = 0
    # to small cut it off
    s_pos[(nh_f ==0) & (nh_s <= min_p)] = 0
    # seems significan keep it
    s_pos[(nh_f ==0) & (nh_s > min_p)] = 1
    
    if draw_hist:    
        if sf_ratio.sum()>0:
            ax[1,1].bar(b[:-1], sf_ratio, width= step, color='blue')
            ax[1,1].set_ylim([0,sf_ratio.max()*scale])
        
        if s_pos.sum()>0:
        #this is scaled to ratio so it is always visible
            ax[1,1].bar(b[:-1], s_pos*ratio, width= step, color='orange')

        ax[1,1].set_title("S/F ratio above "+str(ratio))
        ax[1,1].set_xlim([b[0]*scale,b[-1]*scale])
    
    
    #shorten the arraus and keep only significant values
    short_ratio = np.copy(sf_ratio)
    short_ratio = short_ratio[sf_ratio>=ratio]    
    #-1 for sorting desc
    short_ratio = np.sort(-1 *short_ratio)*-1
    
    #isnt dropout too much
    drop_ok = ((f_dropped< drop_p) and (s_dropped< drop_p)) 
    
    looks_good = False
        
    if (short_ratio.size> 0) and (drop_ok):
        if (cat): #(cat and print_out):
            print "vals: " + str(unique_vals[sf_ratio>=ratio])
        
        print "ratio: " + str(sf_ratio[sf_ratio>=ratio])
        print "suc: " +str(nh_s[sf_ratio>=ratio])
        
        looks_good = True 
        print "sorted ratio: "+str(short_ratio)
        print "***"
            
    #sort only the suc > min_p
    only_suc = s_pos * nh_s
    short_suc = np.copy(only_suc)
    short_suc = short_suc[only_suc>0] 
    #-1 for sorting desc
    short_suc = np.sort(-1 *short_suc)*-1
    
    if (short_suc.size>0) and (drop_ok):
        if (cat): #(cat and print_out):
            print "val: " +str(unique_vals[only_suc>0])
        
        print "no_fail: " +str(nh_s[only_suc>0])
        
        looks_good = True
        print "sorted no_fail: " +str(short_suc)
        print "***"
        
    
    if looks_good:
        print "Dropped values S/F: {0:2.2f}% / {1:2.2f}%".format(s_dropped*100, f_dropped*100)
        print str(index)+" "+ name
        if cat:
            print unique_vals
        
    return looks_good


In [5]:
string_cols = [1, 6, 8]

date_cols=[3, 13, 14]
 
set_cols=[7, 9]

boolean_cols=[2, 12, 24, 27, 29, 30, 31, 32, 34, 35, 38, 46, 47, 48, 49, 50, 51, 52, 53, 54,\
             55, 57, 58, 59, 63, 64, 68, 69, 70, 73, 77, 78, 81, 82, 83, 84, 85, 86, 89, 90,\
             91, 97]

perc_cols = range(102,116)

num_cols=[4, 5, 10, 11, 15, 18, 19, 20, 21, 22, 23, 25, 66, 72, 74, 88, 94, 95, 96, 98, 99,116]

cat_cols=[16, 17, 26, 28, 33, 35, 37, 39, 40, 43, 44, 45, 54, 56, 57, 59, 60, 61, 62, 65,\
          67, 71, 73, 75, 76, 79, 80, 87, 92, 93, 100, 101]


boolean_cols=[2, 12, 24, 27, 29, 30, 31, 32, 34, 36, 38, 41, 42, 46, 47, 48, 49, 50, 51,\
              52, 53, 55, 58, 63, 64, 68, 69, 70, 77, 78, 81, 82, 83, 84, 85, 86, 89,\
              90,91]


In [11]:
print cat_var 
print lower_var
print num_var
print other_var



[2, 3, 4, 12, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 70, 71, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 97, 100, 101, 112, 116]
[12, 26, 39, 40]
[5, 10, 11, 15, 23, 66, 72, 74, 88, 94, 95, 96, 98, 99, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115]
[1, 6, 7, 8, 9, 13, 14, 16, 62]


In [149]:
# is_good = False
# i_col = 0

while(not is_good):
    is_good = survey_column(cat_var[i_col], force_num = False, cat = True,\
                            ratio = 9.0, min_p = 0.20, drop_p = 0.2,\
                            print_out = False, draw_hist = False, print_uvals = False)
    
    if(not is_good):
        i_col+=1
        
i_col+=1
is_good = False

IndexError: list index out of range

In [131]:
# is_good = False
# i_col = 0

while(not is_good):
    is_good = survey_column(num_var[i_col], force_num = True, cat = False,\
                            ratio = 4.0, min_p = 0.05, drop_p = 0.5,\
                            print_out = False, draw_hist = False, print_uvals = True)
    if(not is_good):
        i_col+=1
        
i_col+=1
is_good = False

IndexError: list index out of range

There are 115 (116? variables) for each company. Shockingly there are almost twice as many (305) sucessful companies as failed ones (167). This is of course *VERY SUSPICIOUS* and likely and indication of a synthetic data set. In real life most companies fail.

This my also indicate that we might address the inverse problem, that is 